# Title
Intro

In [ ]:
# Import modules
%matplotlib inline


import os
import sys
import matplotlib

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

from osgeo import gdal
from functools import partial
from pathlib import Path
from pprint import pprint
from math import ceil

gdal.UseExceptions()

In [ ]:
# Map parallel processor location
src = Path.cwd().parent.joinpath("src").as_posix()

sys.path.insert(0, src)

# Import repo modules
from parallelprocessor import ParallelProcessor
from example_workers import gdal_translate

In [ ]:
help(gdal_translate)

Define functions

In [ ]:
def geotiff_to_rgb_array(raster):
    """Read a RGB geotiff and return a np.array which can be plotted by matplotlib."""
    
    src = gdal.Open(raster, 0)

    bands = tuple(
        src.GetRasterBand(i + 1).ReadAsArray()
        for i in range(src.RasterCount)
    )
    
    src = None
    
    arr = np.dstack(bands)
    
    return arr

Set up paths

In [ ]:
examples = Path.cwd()

data = examples.parent.joinpath("data")

tiles = data.joinpath("tiles")

if not tiles.exists():
    tiles.mkdir()

geotiff = data.joinpath("geotiffs", "wellington-03m-rural-aerial-photos-2021.tif")

geotiff_path = geotiff.as_posix()

tile_base = tiles.joinpath(geotiff.stem + "_{}_{}.tif").as_posix()

Preview input

In [ ]:
img_array = geotiff_to_rgb_array(geotiff_path)

plt.imshow(img_array, )

plt.show()

img_array = None

Create pixel offset windows

In [ ]:
# Get raster dimensions
src = gdal.Open(geotiff_path, 0)

width = src.RasterXSize
height = src.RasterYSize

src = None

In [ ]:
x_0 = 0
y_0 = 0
x_step = 256
y_step = 256

f_x_step = lambda i, width, x_step: x_step if i + x_step <= width else width - i
f_y_step = lambda j, height, y_step: y_step if j + y_step <= height else height - j

f = partial(f_x_step, width=width, x_step=x_step)
g = partial(f_y_step, height=height, y_step=y_step)

windows = [
    (
        tile_base.format(
            ceil(i/x_step),
            ceil(j/y_step)
        ), 
        [i, j, f(i), g(j)],
    )
    for i in range(x_0, width, x_step)
    for j in range(y_0, height, y_step)
]

In [ ]:
for tile_name, tile_offsets in windows[:5]:
    print(Path(tile_name).name, tile_offsets)

Create gdal_translate commands

In [ ]:
argument_list = [
    {
        "process_id": tile_path,
        "func_args": (tile_path, geotiff_path),
        "func_kwargs": {
            "format": "COG",
            "srcWin": window,
            "creationOptions": [
                "COMPRESS=WEBP",
                "QUALITY=90",
                "PREDICTOR=YES",
            ],
        }
    }
    for tile_path, window in windows
]

In [ ]:
pprint(argument_list[0])

Initialize parallel processer and set arguments

In [ ]:
parallel_processor = ParallelProcessor(gdal_translate)

for argument_dict in argument_list:
    parallel_processor.add_argument(**argument_dict)

In [ ]:
parallel_processor.run(progressbar=True, timeout=60*10)

In [ ]:
results = parallel_processor.results

Display results

In [ ]:
rasters = {}

x_max = 0
y_max = 0

for raster, status in results.items():
    if status:
        x, y = map(int, Path(raster).stem.rsplit("_")[-2:])
        x_max = x if x > x_max else x_max
        y_max = y if y > y_max else y_max
        rasters[raster] = (x, y)

In [ ]:
fig, axes = plt.subplots(x_max + 1, y_max + 1, figsize=(15, 15))

for raster, (x, y) in rasters.items():
    
    img_array = geotiff_to_rgb_array(geotiff_path)

    ax = axes[y][x]
    
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    
    ax.imshow(img_array)

plt.show()

img_array = None